# Distributed Data

JuliaDB can distributed datasets across multiple processes and can work with data larger than available memory (RAM).

First let's start by adding some worker Julia processes.  If you do not specify a number, `addprocs` will add as many processes as there are CPU cores available.

In [ ]:
addprocs(2)

Before we take advantage of these worker processes, let's load `JuliaDB`. We can also use the command `IndexedTables.set_show_compact!(false)` to make sure that we default to seeing full data tables rather than summaries of their fields.

In [ ]:
using JuliaDB

IndexedTables.set_show_compact!(false)

### When multiple processes are available, `loadtable` will now create distributed tables.

Note the line above the column names printed below: 
```
Distributed Table with 56023 rows in 2 chunks
```

In [ ]:
dt = loadtable("stocksample", filenamecol = :Ticker, indexcols = [:Ticker, :Date])

#### Notable difference 1: No `getindex`

Now that we're working with a distributed table, indexing will no longer work.

In [ ]:
dt[1]

#### Notable difference 2: Not iterable

Similarly, we're not able to iterate over the contents of a distributed table.

In [ ]:
for row in dt
    println(row)
end

### Bring Distributed Table Into Master Process

While not necessary for most operations, you may occasionally want to bring a dataset into the master process. This will allow you to, for example, iterate over the distributed table's rows.  This is accomplished with the `collect` function.

Note that after `collect`ing, the table's header says `Table` instead of `Distributed Table`.

In [ ]:
t = collect(dt)

### Queries still work on a distributed table!

#### Functions that return a single value still return a single value.

In [ ]:
reduce(+, dt; select = :Close)

#### Functions that returned a Table now return a Distributed Table.

In [ ]:
groupreduce(+, dt, :Ticker; select = :Close)

#### Functions that returned an `Array` now return a `DArray` (distributed array).

In [ ]:
select(dt, :Close)

#### Just like for tables, `collect` will change a `DArray` to an `Array`:

In [ ]:
collect(select(dt, :Close))

### Out-of-Core Functionality

Reference: http://juliadb.org/latest/manual/out-of-core.html

JuliaDB can be used to load/query datasets that are too big to fit in memory (RAM).

#### How this works:

*Data is loaded into a distributed dataset in *chunks* that fit in memory.*

The `loadtable` and `loadndsparse` functions take an `output` keyword argument which can be set to a directory where the loaded data is written to in an efficient binary format.

In [ ]:
loadtable("stocksample", output = "bin", filenamecol=:Ticker, indexcols=[:Ticker, :Date])

### Everything below is a WIP

1. Data is loaded into a distributed dataset containing *chunks* that are small enough to fit in memory.
1. Data is processed `p` chunks at a time – where `p` is the number of worker processes. This means `p * size of chunks` should fit in memory!
1. Output data (from `reduce`, etc.) is accumulated in-memory and must be small enough to fit in the available memory.

Several queries are designed to work on such datasets:

1. `reduce`
1. `groupreduce`

#### Loading Data Out-of-Core

Out-of-core processing is achieved using the `output` and `chunks` keyword arguments. For example,

In [ ]:
loadtable("stocksample", output = "bin", chunks = 8,
    filenamecol=:Ticker, indexcols=[:Ticker, :Date])

### Plotting Big Data

#### StatPlots

The StatPlots package integrates with Plots to plot a variety of tabular data structures, including those in JuliaDB.

In [ ]:
using StatPlots
gr()

Normally we could generate a plot simply by calling the `plot` function. When working with tables (as well as DataFrames, DataStreams, etc.), we need to precede our call to `plot` with the `@df` macro and the name of the table. This @df command allows the plot call to manipulate a table's columns as symbols as if they were `Array`s.

For example, the syntax
```julia
@df tablename plot(:x, :y)
```
allows us to plot the columns `x` and `y` of the table `tablename`:

In [ ]:
tablename = table(@NT(x = [1, 2, 3], y = [1, 4, 9]))
@df tablename plot(:x, :y)

To plot a distributed table with columns `x` and `y`, we would use the syntax

```julia
@df collect(tablename) plot(:x, :y)
```

To plot our stock data currently stored in the distributed table, `dt`, by stock (`Ticker` field), we use the `group` keyword argument to `plot`:

In [ ]:
@df collect(dt) plot(:Date, :Close; group=:Ticker, legend=:topleft)

#### Partition plots

If you are trying to distribute your data in the first place, it's probably big. In this case, you may want to use `partitionplot` which

- recognizes that when data is huge it doesn't make sense to plot every point.
- incrementally builds summaries of the data.  
- relies on fixed-size memory data structures that can handle infinite data streams.

**We'll see more on this in the next notebook, but for now let's try it out!**

The syntax is

```julia
partitionplot(tablename, :x, :y)
```

for a table, `tablename`, with columns `x` and `y`.

Note that here we need neither the `@df` macro nor a call to `collect` (for distributed tables).

For example, the following command plots the data from `dt` by stock (`Ticker` field) using the `by` keyword argument:

In [ ]:
using Plots

partitionplot(dt, :Date, :Close; by=:Ticker, legend=:topleft)